In [203]:
## Code from https://towardsdatascience.com/linear-regression-with-pytorch-eb6dedead817
## Code from https://machinelearningmastery.com/making-predictions-with-multilinear-regression-in-pytorch/

In [204]:
## import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

## EDA

In [205]:
## import data from the master folder
data = pd.read_csv(r'C:\Users\Siyang\Documents\GitHub\Fantasy-Premier-League\data\2023-24\gws\merged_gw.csv')

In [206]:
data.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Femi Seriki,DEF,Sheffield Utd,0.5,0,0,0,0,0.0,653,...,0,0.0,0,0,0,0,40,True,0,1
1,Jack Hinshelwood,MID,Brighton,1.5,0,0,0,0,0.0,621,...,4,0.0,0,0,0,0,45,True,0,1
2,Jadon Sancho,MID,Man Utd,3.0,0,0,4,0,11.3,397,...,1,8.0,1,0,0,0,70,True,0,1
3,Rhys Norrington-Davies,DEF,Sheffield Utd,0.1,0,0,0,0,0.0,487,...,0,0.0,0,0,0,0,40,True,0,1
4,Vitaly Janelt,MID,Brentford,2.1,0,0,6,0,11.5,105,...,2,17.0,2,0,0,0,55,True,0,1


In [258]:
selected_gameweeks_train = [1,2,3,4,5] ## filter by round in database
selected_gameweeks_test = [6,7,8,9,10]
filter_by_gw = data[data['round'].isin(selected_gameweeks_train + selected_gameweeks_test)]
filter_by_gw

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,Femi Seriki,DEF,Sheffield Utd,0.5,0,0,0,0,0.0,653,...,0,0.0,0,0,0,0,40,True,0,1
1,Jack Hinshelwood,MID,Brighton,1.5,0,0,0,0,0.0,621,...,4,0.0,0,0,0,0,45,True,0,1
2,Jadon Sancho,MID,Man Utd,3.0,0,0,4,0,11.3,397,...,1,8.0,1,0,0,0,70,True,0,1
3,Rhys Norrington-Davies,DEF,Sheffield Utd,0.1,0,0,0,0,0.0,487,...,0,0.0,0,0,0,0,40,True,0,1
4,Vitaly Janelt,MID,Brentford,2.1,0,0,6,0,11.5,105,...,2,17.0,2,0,0,0,55,True,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7061,Kyle Walker,DEF,Man City,4.5,0,2,39,1,26.6,369,...,0,4.0,8,-80597,39634,120231,53,False,0,10
7062,Jacob Brown,FWD,Luton,1.0,0,0,-1,0,0.6,631,...,3,0.0,1,-712,500,1212,49,False,0,10
7063,Vicente Guaita,GK,Crystal Palace,0.0,0,0,0,0,0.0,227,...,1,0.0,0,-2129,0,2129,44,True,0,10
7064,Braian Ojeda Rodríguez,MID,Nott'm Forest,0.0,0,0,0,0,0.0,459,...,3,0.0,0,-11,0,11,45,False,0,10


In [250]:
selected_attributes = ['element','goals_scored', 'assists',
       'minutes', 'goals_conceded', 'creativity', 'influence', 'threat',
       'bonus', 'bps', 'ict_index', 'clean_sheets', 'red_cards',
       'yellow_cards','total_points','round']

In [251]:
filter_by_gw_attributes = filter_by_gw[selected_attributes]
filter_by_gw_attributes

,element,goals_scored,assists,minutes,goals_conceded,creativity,influence,threat,bonus,bps,ict_index,clean_sheets,red_cards,yellow_cards,total_points,round
0,653,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,1
1,621,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,1
2,397,0,0,22,0,11.3,3.8,8.0,0,4,2.3,0,0,0,1,1
3,487,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,1
4,105,0,0,90,2,11.5,14.6,17.0,0,6,4.3,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7061,369,0,0,90,0,26.6,24.4,4.0,2,39,5.5,1,0,0,8,10
7062,631,0,0,57,2,0.6,6.6,0.0,0,-1,0.7,0,0,0,1,10
7063,227,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,10
7064,459,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,10


In [252]:
filter_by_gw_attributes.shape

(7066, 16)

In [265]:
# train_input_tensor = np.zeros((max(filter_by_gw_attributes['element']), len(selected_gameweeks), filter_by_gw_attributes.shape[1]-3))
train_player_count = max(filter_by_gw_attributes[filter_by_gw_attributes['round'].isin(selected_gameweeks_train)]['element'])
test_player_count = max(filter_by_gw_attributes[filter_by_gw_attributes['round'].isin(selected_gameweeks_test)]['element'])

train_inputs = np.zeros((train_player_count, len(selected_gameweeks_train) - 1))
train_labels = np.zeros(train_player_count)

test_inputs = np.zeros((test_player_count, len(selected_gameweeks_test) - 1))
test_labels = np.zeros(test_player_count)

for i in tqdm(range(filter_by_gw_attributes.shape[0])):
    for j in range(filter_by_gw_attributes.shape[1]-1):
        for k in range(len(selected_gameweeks_train) + len(selected_gameweeks_test)):

            ## select player
            element = int(filter_by_gw_attributes.iloc[i][0]) - 1 # element indexing starts from 1

            ## select the gameweek of that row
            gw = filter_by_gw_attributes.loc[i]['round']

            ## get points earned by that player in that week
            total_points = filter_by_gw_attributes.iloc[i][-2]

            ## enter the total points into the right dataset (train/test, input/label)
            if gw in selected_gameweeks_train: # if it is last round, thats data that we are supposed to predict
                gw_index = int(gw - min(selected_gameweeks_train))
                if gw == selected_gameweeks_train[-1]:
                    train_labels[element] = total_points
                else:
                    train_inputs[element][gw_index] = total_points
            elif gw in selected_gameweeks_test:
                gw_index = int(gw - min(selected_gameweeks_test))
                if gw == selected_gameweeks_test[-1]:
                    test_labels[element] = total_points
                else:
                    test_inputs[element][gw_index] = total_points

100%|██████████| 7066/7066 [02:32<00:00, 46.23it/s]


In [266]:
print(test_inputs.shape)
print(test_labels.shape)

(732, 4)
(732,)


## Model Training

In [267]:
import torch # torch will allow us to create tensors.
import torch.nn as nn # torch.nn allows us to create a neural network.
import torch.nn.functional as F # nn.functional give us access to the activation and loss functions.
from torch.optim import Adam # optim contains many optimizers. This time we're using Adam

import lightning as L # lightning has tons of cool tools that make neural networks easier
from torch.utils.data import TensorDataset, DataLoader # these are needed for the training data

## Set the seed so that, hopefully, everyone will get the same results as me.
from pytorch_lightning.utilities.seed import seed_everything

In [275]:
train_inputs, train_labels = (torch.tensor(train_inputs), torch.tensor(train_labels))
train_inputs = train_inputs.to(torch.float32).to('cuda')
train_labels = train_labels.to(torch.float32).to('cuda')

test_inputs, test_labels = (torch.tensor(test_inputs), torch.tensor(test_labels))
test_inputs = test_inputs.to(torch.float32).to('cuda')
test_labels = test_labels.to(torch.float32).to('cuda')

C:\Users\siyang\AppData\Local\Temp\ipykernel_12576\4047564240.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs, train_labels = (torch.tensor(train_inputs), torch.tensor(train_labels))


In [281]:
train_dataset = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader(train_dataset)

test_dataset = TensorDataset(test_inputs, test_labels)
test_dataloader = DataLoader(test_dataset)

In [283]:
## Instead of coding an LSTM by hand, let's see what we can do with PyTorch's nn.LSTM()
class LightningLSTM(L.LightningModule):

    def __init__(self): # __init__() is the class constructor function, and we use it to initialize the Weights and Biases.
        
        super().__init__() # initialize an instance of the parent class, LightningModule.

        seed_everything(seed=42)
        
        ## input_size = number of features (or variables) in the data. In our example
        ##              we only have a single feature (value)
        ## hidden_size = this determines the dimension of the output
        ##               in other words, if we set hidden_size=1, then we have 1 output node
        ##               if we set hiddeen_size=50, then we hve 50 output nodes (that can then be 50 input
        ##               nodes to a subsequent fully connected neural network.
        self.lstm = nn.LSTM(input_size=1, hidden_size=1) 
         

    def forward(self, input):
        ## transpose the input vector
        input_trans = input.view(len(input), 1)
        
        lstm_out, temp = self.lstm(input_trans)
        
        ## lstm_out has the short-term memories for all inputs. We make our prediction with the last one
        prediction = lstm_out[-1] 
        return prediction
        
        
    def configure_optimizers(self): # this configures the optimizer we want to use for backpropagation.
        return Adam(self.parameters(), lr=0.001) ## we'll just go ahead and set the learning rate to 0.1

    
    def training_step(self, batch, batch_idx): # take a step during gradient descent.
        input_i, label_i = batch # collect input
        output_i = self.forward(input_i[0]) # run input through the neural network
        loss = (output_i - label_i)**2 ## loss = squared residual
        
        ###################
        ##
        ## Logging the loss and the predicted values so we can evaluate the training
        ##
        ###################
        self.log("train_loss", loss)
        
        if (label_i == 0):
            self.log("out_0", output_i)
        else:
            self.log("out_1", output_i)

        return loss
    
    ## from tutorial
    ## https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.core.LightningModule.html#lightning.pytorch.core.LightningModule.test_step
    def test_step(self, batch, batch_idx):
        x, y = batch

        # implement your own
        out = self(x[0])
        loss = (out - y)**2 ## loss = squared residual

        # log 6 example images
        # or generated text... or whatever
        # sample_imgs = x[:6]
        # grid = torchvision.utils.make_grid(sample_imgs)
        # self.logger.experiment.add_image('example_images', grid, 0)

        # calculate acc
        # labels_hat = torch.argmax(out, dim=1)
        # test_acc = torch.sum(y == labels_hat).item() / (len(y) * 1.0)

        # log the outputs!
        self.log_dict({'test_loss': loss})

In [284]:
model = LightningLSTM() # First, make model from the class

## print out the name and value for each parameter
print("Before optimization, the parameters are...")
for name, param in model.named_parameters():
    print(name, param.data)
    
print("\nNow let's compare the observed and predicted values...")
print("Company A: Observed = 0, Predicted =", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())
print("Company B: Observed = 1, Predicted =", model(torch.tensor([1., 0.5, 0.25, 1.])).detach())

c:\Users\siyang\Documents\GitHub\data-driven-fpl\.venv\Lib\site-packages\pytorch_lightning\utilities\seed.py:48: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v1.10.0. Please use `lightning_lite.utilities.seed.seed_everything` instead.
Global seed set to 42


Before optimization, the parameters are...
lstm.weight_ih_l0 tensor([[ 0.7645],
        [ 0.8300],
        [-0.2343],
        [ 0.9186]])
lstm.weight_hh_l0 tensor([[-0.2191],
        [ 0.2018],
        [-0.4869],
        [ 0.5873]])
lstm.bias_ih_l0 tensor([ 0.8815, -0.7336,  0.8692,  0.1872])
lstm.bias_hh_l0 tensor([ 0.7388,  0.1354,  0.4822, -0.1412])

Now let's compare the observed and predicted values...
Company A: Observed = 0, Predicted = tensor([0.6675])
Company B: Observed = 1, Predicted = tensor([0.6665])


In [285]:
## NOTE: Because we have set Adam's learning rate to 0.1, we will train much, much faster.
## Before, with the hand made LSTM and the default learning rate, 0.001, it took about 5000 epochs to fully train
## the model. Now, with the learning rate set to 0.1, we only need 300 epochs. Now, because we are doing so few epochs,
## we have to tell the trainer add stuff to the log files every 2 steps (or epoch, since we have to rows of training data)
## because the default, updating the log files every 50 steps, will result in a terrible looking graphs. So
if torch.cuda.is_available():
   model.cuda()

trainer = L.Trainer(max_epochs=55, log_every_n_steps=718)

trainer.fit(model, train_dataloaders=train_dataloader)

print("After optimization, the parameters are...")
for name, param in model.named_parameters():
    print(name, param.data)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type | Params
------------------------------
0 | lstm | LSTM | 16    
------------------------------
16        Trainable params
0         Non-trainable params
16        Total params
0.000     Total estimated model params size (MB)
c:\Users\siyang\Documents\GitHub\data-driven-fpl\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 0:   2%|▏         | 11/718 [00:00<00:05, 139.25it/s, v_num=11]

Epoch 54: 100%|██████████| 718/718 [00:02<00:00, 248.82it/s, v_num=11]

`Trainer.fit` stopped: `max_epochs=55` reached.


Epoch 54: 100%|██████████| 718/718 [00:02<00:00, 248.13it/s, v_num=11]
After optimization, the parameters are...
lstm.weight_ih_l0 tensor([[3.7138],
        [1.8885],
        [1.9305],
        [2.9134]])
lstm.weight_hh_l0 tensor([[0.1456],
        [2.2723],
        [0.2165],
        [3.1459]])
lstm.bias_ih_l0 tensor([-0.3195, -0.4352,  0.3627,  0.7719])
lstm.bias_hh_l0 tensor([-0.4623,  0.4339, -0.0243,  0.4436])


## Result Analysis

In [288]:
print("\nNow let's compare the observed and predicted values...")
element_0 = 10
element_1 = 395
element_2 = 354
element_3 = 307

if torch.cuda.is_available():
   model.cuda()

print(f"Player id {element_0 + 1}: Observed = {test_labels[element_0]}, Predicted =", model(test_inputs[element_0]).detach())
print(f"Player id {element_1 + 1}: Observed = {test_labels[element_1]}, Predicted =", model(test_inputs[element_1]).detach())
print(f"Player id {element_2 + 1}: Observed = {test_labels[element_2]}, Predicted =", model(test_inputs[element_2]).detach()) ##haaland
print(f"Player id {element_3 + 1}: Observed = {test_labels[element_3]}, Predicted =", model(test_inputs[element_3]).detach()) ##salah


Now let's compare the observed and predicted values...
Player id 11: Observed = 0.0, Predicted = tensor([0.1980], device='cuda:0')
Player id 396: Observed = 2.0, Predicted = tensor([0.9993], device='cuda:0')
Player id 355: Observed = 16.0, Predicted = tensor([0.9993], device='cuda:0')
Player id 308: Observed = 8.0, Predicted = tensor([0.9992], device='cuda:0')


In [289]:
trainer.test(model,dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\siyang\Documents\GitHub\data-driven-fpl\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 732/732 [00:01<00:00, 685.03it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss            4.829298496246338
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 4.829298496246338}]